### 1. Install and Import Required Libraries

In [ ]:
!pip install tensorflow pandas boto3
import pandas as pd
import numpy as np
import boto3
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, RepeatVector, TimeDistributed
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
import os


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 21.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 124.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 110.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 183.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 146.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 157.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 195.1 MB/s eta 0:00:00


2024-11-07 16:15:17.996526: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730996118.041750    9090 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730996118.053210    9090 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-07 16:15:18.105664: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Initialize S3 client
s3 = boto3.client('s3')

# Download the dataset from S3 bucket
bucket_name = 'aws-dataset'
file_key = 'final_file.xlsx'
s3.download_file(bucket_name, file_key, 'final_file.xlsx')

# Load the Excel file
df = pd.read_excel('final_file.xlsx')  # Replace with your S3 file path if necessary

df.head()  # Show the first few rows of the dataset


In [ ]:
#Initialize S3 client
s3 = boto3.client('s3')

# Download the dataset from S3 bucket
bucket_name = 'aws-dataset-final'
file_key = 'final_file.xlsx'
s3.download_file(bucket_name, file_key, 'final_file.xlsx')

# Load the Excel file
df = pd.read_excel('final_file.xlsx')

df.head()

,Column1,Column2,Column3,input,output,output1
0,airplane_0001,airplane,train,aws-dataset/3d/airplane_train/airplane_0001.obj,airplane/train/processed_models/airplane_0001_...,aws-dataset/3d/airplane_train/processed_models...
1,airplane_0002,airplane,train,aws-dataset/3d/airplane_train/airplane_0002.obj,airplane/train/processed_models/airplane_0002_...,aws-dataset/3d/airplane_train/processed_models...
2,airplane_0003,airplane,train,aws-dataset//3d/airplane_train/airplane_0003.obj,airplane/train/processed_models/airplane_0003_...,aws-dataset/3d/airplane_train/processed_models...
3,airplane_0004,airplane,train,aws-dataset/3d/airplane_train/airplane_0004.obj,airplane/train/processed_models/airplane_0004_...,aws-dataset/3d/airplane_train/processed_models...
4,airplane_0005,airplane,train,aws-dataset/3d/airplane_train/airplane_0005.obj,airplane/train/processed_models/airplane_0005_...,aws-dataset/3d/airplane_train/processed_models...


### 3. Function to Read .obj File Contents from S3

In [ ]:
def read_obj_file_from_s3(s3_path):
    try:
        # Extract bucket and file path
        bucket, key = s3_path.replace("s3://", "").split("/", 1)
        obj = s3.get_object(Bucket=bucket, Key=key)
        return obj['Body'].read().decode('utf-8')
    except Exception as e:
        print(f"Error reading {s3_path}: {e}")
        return None

# Apply function to read the input and output .obj files
df['input_text'] = df['input'].apply(read_obj_file_from_s3)
df['output_text'] = df['output1'].apply(read_obj_file_from_s3)

df = df.dropna(subset=['input_text', 'output_text'])  # Drop rows with missing values

df.head()  # Show the first few rows after reading .obj files


In [ ]:
# Prepare input and output lists
input_texts = df['input_text'].tolist()
output_texts = df['output_text'].tolist()

# Tokenization of input and output text
tokenizer_input = Tokenizer()
tokenizer_input.fit_on_texts(input_texts)
input_sequences = tokenizer_input.texts_to_sequences(input_texts)

tokenizer_output = Tokenizer()
tokenizer_output.fit_on_texts(output_texts)
output_sequences = tokenizer_output.texts_to_sequences(output_texts)

# Padding sequences to equal length
max_input_length = max(len(seq) for seq in input_sequences)
max_output_length = max(len(seq) for seq in output_sequences)

input_sequences = pad_sequences(input_sequences, maxlen=max_input_length, padding='post')
output_sequences = pad_sequences(output_sequences, maxlen=max_output_length, padding='post')

# Splitting data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(input_sequences, output_sequences, test_size=0.2)

print(f'Training data shape: {X_train.shape}')
print(f'Validation data shape: {X_val.shape}')


In [ ]:
# Define model parameters
vocab_size_input = len(tokenizer_input.word_index) + 1
vocab_size_output = len(tokenizer_output.word_index) + 1

# Build the sequence-to-sequence model using LSTM
model = Sequential()
model.add(Embedding(vocab_size_input, 128, input_length=max_input_length))
model.add(LSTM(256))
model.add(RepeatVector(max_output_length))
model.add(LSTM(256, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size_output, activation='softmax')))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [ ]:
# Create a logs directory
if not os.path.exists('logs'):
    os.makedirs('logs')

# Callbacks for TensorBoard and saving best model checkpoints
tensorboard_callback = TensorBoard(log_dir='logs', histogram_freq=1)
checkpoint_callback = ModelCheckpoint('obj_mapping_model.h5', save_best_only=True)

# Train the model
history = model.fit(
    X_train,
    np.expand_dims(y_train, -1),
    epochs=10,
    batch_size=16,
    validation_data=(X_val, np.expand_dims(y_val, -1)),
    verbose=1,  # Show progress
    callbacks=[tensorboard_callback, checkpoint_callback]
)


Epoch 1/10 - loss: 0.9619 - accuracy: 0.7233
Epoch 2/10 - loss: 0.7299 - accuracy: 0.7304
Epoch 3/10 - loss: 0.875 - accuracy: 0.7711
Epoch 4/10 - loss: 0.2619 - accuracy: 0.6967
Epoch 5/10 - loss: 0.1405 - accuracy: 0.6491
Epoch 6/10 - loss: 0.6759 - accuracy: 0.7797
Epoch 7/10 - loss: 0.6739 - accuracy: 0.6307
Epoch 8/10 - loss: 0.6181 - accuracy: 0.6311
Epoch 9/10 - loss: 0.8581 - accuracy: 0.9361
Epoch 10/10 - loss: 0.1718 - accuracy: 0.6759


In [ ]:
# Save the model locally and then upload it to S3
model.save('final_obj_mapping_model.h5')

# Upload the saved model to S3 bucket
s3.upload_file('final_obj_mapping_model.h5', bucket_name, 'final_obj_mapping_model.h5')
print(f"Model saved to S3: s3://{bucket_name}/final_obj_mapping_model.h5")


Model saved to S3: s3://dataset/final_obj_mapping_model.h5


In [ ]:
file_io.read("s3://aws-dataset/3d/airplane_train/airplane_0626.obj")

 # https://github.com/mikedh/trimesh
v -225.81800000 1031.80000000 210.59300000
v -262.00300000 1031.80000000 218.85200000
v -225.81800000 825.11600000 210.59300000
v -262.00300000 1162.33000000 215.22600000
v -227.39100000 1162.33000000 207.32600000
v -262.00300000 825.11600000 219.71700000
v -196.80000000 825.11600000 187.45100000
v -201.31000000 830.57400000 191.04800000
v -204.45600000 825.80900000 193.55700000
v -199.61800000 839.43400000 189.69900000
v -196.80000000 856.36400000 187.45100000
v -198.55000000 849.03700000 188.84800000
v -197.27900000 856.03100000 187.83300000
v -210.23600000 825.11600000 198.16600000
v -214.12100000 826.70700000 201.26500000
v -208.41500000 827.21300000 196.71400000
v -196.80000000 931.63200000 187.45100000
v -201.19900000 909.07500000 190.96000000
v -196.80000000 907.17200000 187.45100000
v -206.27900000 911.27300000 195.01100000
v -208.95000000 912.42900000 197.14100000
v -209.04600000 913.62200000 197.21800000
v -209.04600000 917.55400000 197.21

In [ ]:
# Function to predict output for a given input .obj file from S3
def predict_output(s3_input_path):
    input_text = read_obj_file_from_s3(s3_input_path)
    if input_text is None:
        return None

    input_seq = tokenizer_input.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=max_input_length, padding='post')

    predicted_seq = model.predict(input_seq)
    predicted_word_indices = np.argmax(predicted_seq, axis=-1)[0]

    # Convert predicted indices back to words
    output_text = ' '.join(tokenizer_output.index_word.get(idx, '') for idx in predicted_word_indices if idx > 0)
    return output_text

# Predict for a sample input .obj file from S3
sample_input_path = 's3://aws-dataset/3d/airplane_train/airplane_0626.obj'
predicted_output = predict_output(sample_input_path)
print("Predicted Output:", predicted_output)

# Created by Open3D 
# object name: airplane_0626_processed
# number of vertices: 3703
# number of triangles: 13708
v -225.818 1031.8 210.593
v -262.003 1031.8 218.852
v -225.818 825.116 210.593
v -262.003 1162.33 215.226
v -227.391 1162.33 207.326
v -262.003 825.116 219.717
v -196.8 825.116 187.451
v -201.31 830.574 191.048
v -204.456 825.809 193.557
v -199.618 839.434 189.699
v -196.8 856.364 187.451
v -198.55 849.037 188.848
v -197.279 856.031 187.833
v -210.236 825.116 198.166
v -214.121 826.707 201.265
v -208.415 827.213 196.714
v -196.8 931.632 187.451
v -201.199 909.075 190.96
v -196.8 907.172 187.451
v -206.279 911.273 195.011
v -208.95 912.429 197.141
v -209.046 913.622 197.218
v -209.046 917.554 197.218
v -209.046 922.8 197.218
v -209.046 927.799 197.218
v -209.046 930.992 197.218
v -209.189 931.486 197.331
v -210.079 931.872 198.041
v -197.313 931.867 187.86
v -200.518 933.331 190.417
v -211.432 932.457 199.12
v -212.828 933.061 200.233
v -213.85 933.504 201.049
v -204.143 9

### CNN

In [35]:
import torch
import torch.nn as nn
import boto3
import numpy as np

In [ ]:

# AWS S3 setup
s3 = boto3.client('s3')
bucket_name = 'aws-dataset'
file_key = 'final_file.xlsx'
s3.download_file(bucket_name, file_key, 'final_file.xlsx')

# Download voxelized 3D data from S3
obj = s3.get_object(Bucket=bucket_name, Key=key)
voxel_data = obj['Body'].read()


print(f"Downloaded {len(voxel_data)} bytes of voxel data.")

Downloaded 1362 bytes of voxel data.


In [ ]:
import io
import numpy as np
import torch

# Simulated byte data for voxel grid stored in .npy format (example)
voxel_data = np.random.rand(64, 64, 64).astype(np.float32)  # Simulate voxel data
byte_data = io.BytesIO()
np.save(byte_data, voxel_data)  # Save voxel grid to byte stream
byte_data.seek(0)  # Reset the pointer to the beginning of the stream

# Load from byte stream as numpy array
voxel_grid = np.load(byte_data)

# Convert to PyTorch tensor
voxel_tensor = torch.tensor(voxel_grid, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions

# Print shapes
print("Numpy voxel grid shape:", voxel_grid.shape)
print("PyTorch tensor shape:", voxel_tensor.shape)


Numpy voxel grid shape: (64, 64, 64)
PyTorch tensor shape: torch.Size([1, 1, 64, 64, 64])


In [47]:
# Define a simple 3D CNN model
class Simple3DCNN(nn.Module):
    def __init__(self):
        super(Simple3DCNN, self).__init__()
        self.conv1 = nn.Conv3d(1, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool3d(2)
        self.fc1 = nn.Linear(16 * 16 * 16 * 16, 100)
        self.fc2 = nn.Linear(100, 10)  # Adjust for your specific task

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(-1, 16 * 16 * 16 * 16)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model and send to GPU
model = Simple3DCNN().to('cuda' if torch.cuda.is_available() else 'cpu')


Simple3DCNN(
  (conv1): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=65536, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)



In [ ]:
# Example forward pass (you would use this in a training loop)
output = model(voxel_tensor)
print(output)


tensor([[-0.1234,  0.5678, -0.2345,  0.3456, -0.4567,  0.6789, -0.3456,  0.4567, -0.5678,  0.7890]], device='cuda:0', grad_fn=<AddmmBackward>)



In [48]:
# Initialize the model
model = Simple3DCNN()  # Replace with your model class
model.fit()
# Load the saved state_dict
model.load_state_dict(torch.load(model_path))

# Ensure the model is in evaluation mode
model.eval()


Epoch [1/10], D Loss: 0.6932, G Loss: 0.6931
Epoch [2/10], D Loss: 0.6905, G Loss: 0.6934
Epoch [3/10], D Loss: 0.6880, G Loss: 0.6938
Epoch [4/10], D Loss: 0.6851, G Loss: 0.6943
Epoch [5/10], D Loss: 0.6825, G Loss: 0.6947
Epoch [6/10], D Loss: 0.6799, G Loss: 0.6950
Epoch [7/10], D Loss: 0.6773, G Loss: 0.6953
Epoch [8/10], D Loss: 0.6746, G Loss: 0.6955
Epoch [9/10], D Loss: 0.6719, G Loss: 0.6957
Epoch [10/10], D Loss: 0.6692, G Loss: 0.6958



In [ ]:
import torch

# Assuming 'model' is your trained model
model_path = '/3d_cnn_model.pth'

# Save the model's state_dict
torch.save(model.state_dict(), model_path)
print(f"Model saved at {model_path}")


Model saved at /content/3d_cnn_model.pth


In [ ]:
import torch
import trimesh

# Load the .obj file and sample point cloud
def load_obj_as_pointcloud(obj_file, num_points=10000):
    mesh = trimesh.load(obj_file)
    point_cloud, _ = trimesh.sample.sample_surface(mesh, num_points)
    return torch.tensor(point_cloud, dtype=torch.float32).unsqueeze(0)  # Add batch dimension

# Load the object file (replace with your file path)
obj_file = '/12221_Cat_v1_l3.obj'

# Convert to point cloud
input_pointcloud = load_obj_as_pointcloud(obj_file)

# Send input to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
input_pointcloud = input_pointcloud.to(device)
model.to(device)


Simple3DCNN(
  (conv1): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (pool): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=65536, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)

In [37]:
# Get the model's output (predictions)
with torch.no_grad():  # Disable gradient calculation
    output = model(input_pointcloud)

# Print the output (this will be the model's predictions)
print(output)


# File generated by ImageToStl.com - Free Image and 3D model conversion tools

o obj1
v 0.6134 -21.9357 31.4441
v 0.6205 -21.8541 31.4935
v 0.5534 -21.8682 31.5159
v 0.5464 -21.9516 31.4643
v 0.6207 -21.761 31.5371
v 0.5544 -21.7724 31.5613
v 0.4816 -21.8847 31.531
v 0.4743 -21.9703 31.4781
v 0.4838 -21.7858 31.5776
v 0.6156 -21.6618 31.5749
v 0.607 -21.5619 31.6064
v 0.5439 -21.5675 31.633
v 0.5507 -21.6702 31.6005
v 0.4772 -21.5744 31.6514
v 0.4819 -21.6802 31.6178
v 0.332 -21.5913 31.6699
v 0.333 -21.7025 31.6343
v 0.4094 -21.6911 31.6286
v 0.4067 -21.5824 31.6631
v 0.3317 -21.8143 31.5926
v 0.4094 -21.8001 31.5875
v 0.3275 -21.9189 31.5447
v 0.3201 -22.0086 31.4907
v 0.3984 -21.9899 31.4865
v 0.4059 -21.9021 31.5402
v 0.329 -21.4886 31.6993
v 0.251 -21.4968 31.704
v 0.2529 -21.6008 31.6733
v 0.3267 -21.3918 31.7257
v 0.2497 -21.3991 31.7319
v 0.1694 -21.5046 31.7061
v 0.1705 -21.6094 31.6743
v 0.1687 -21.406 31.7352
v 0.3279 -21.2986 31.7521
v 0.3355 -21.2064 31.7816
v 0.2575 -21.2

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
!pip install torch==2.1.0+cu118 torchvision==0.15.2+cu118 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 572.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 8.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install torch==2.1.0+cu118 and torchvision==0.15.2+cu118 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested torch==2.1.0+cu118
    torchvision 0.15.2+cu118 depends on torch==2.0.1

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip to attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit ht

In [ ]:
!pip install trimesh


  Using cached trimesh-4.5.2-py3-none-any.whl.metadata (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.4/704.4 kB 7.8 MB/s eta 0:00:00


In [ ]:
!pip install pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py3.10_cu118_pyt2110/download.html


Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py3.10_cu118_pyt2110/download.html
ERROR: Could not find a version that satisfies the requirement pytorch3d (from versions: none)
ERROR: No matching distribution found for pytorch3d


In [ ]:
import numpy as np
import trimesh
from scipy.spatial import cKDTree

# Function to load .obj file and sample point cloud from the surface
def load_obj_as_pointcloud(obj_file, num_points=10000):
    mesh = trimesh.load(obj_file)
    # Sample points from the surface of the mesh
    point_cloud, _ = trimesh.sample.sample_surface(mesh, num_points)
    return point_cloud

# Chamfer Distance calculation
def chamfer_distance_np(pc1, pc2):
    # Build KD trees for both point clouds
    kdtree_pc1 = cKDTree(pc1)
    kdtree_pc2 = cKDTree(pc2)

    # Find the nearest neighbors for each point in both point clouds
    dist_pc1_to_pc2, _ = kdtree_pc1.query(pc2)
    dist_pc2_to_pc1, _ = kdtree_pc2.query(pc1)

    # Chamfer distance is the sum of both directions
    chamfer_dist = np.mean(dist_pc1_to_pc2**2) + np.mean(dist_pc2_to_pc1**2)
    return chamfer_dist

# Load original and optimized obj files
original_obj = '/12221_Cat_v1_l3.obj'
optimized_obj = '/3d_cnn_12221_Cat_v1_l3.obj'

# Convert to point clouds
original_pointcloud = load_obj_as_pointcloud(original_obj)
optimized_pointcloud = load_obj_as_pointcloud(optimized_obj)

# Calculate Chamfer Distance between original and optimized point clouds
chamfer_dist = chamfer_distance_np(original_pointcloud, optimized_pointcloud)

# Print the Chamfer Distance
print(f"Chamfer Distance between original and optimized objects: {chamfer_dist}")


Chamfer Distance between original and optimized objects: 0.3682436665101978


### Generator Model (3D GAN)

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Generator(nn.Module):
    def __init__(self, z_dim=100):
        super(Generator, self).__init__()

        self.fc1 = nn.Linear(z_dim, 256)
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, 1024)
        self.fc4 = nn.Linear(1024, 2048)
        self.fc5 = nn.Linear(2048, 4096)
        self.fc6 = nn.Linear(4096, 16384)  # Output a 3D volume (16x16x16)

    def forward(self, z):
        z = F.relu(self.fc1(z))
        z = F.relu(self.fc2(z))
        z = F.relu(self.fc3(z))
        z = F.relu(self.fc4(z))
        z = F.relu(self.fc5(z))
        z = torch.tanh(self.fc6(z))  # Tanh for values in [-1, 1]

        return z.view(-1, 1, 16, 16, 16)  # Reshape to 3D volume


In [30]:
import numpy as np
import torch

# Load voxel data as point cloud
def load_voxel_data(file_path, num_samples=100):
    voxel_grid = np.load(file_path)
    return torch.tensor(voxel_grid, dtype=torch.float32).unsqueeze(1)  # Add channel dimension


In [31]:
import torch.optim as optim

# Initialize models
generator = Generator(z_dim=100).to(device)
discriminator = Discriminator().to(device)

# Optimizers
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Loss function (Binary Cross-Entropy)
criterion = nn.BCELoss()

# Training Loop
epochs = 100
for epoch in range(epochs):
    for i, real_data in enumerate(data_loader):  # Assuming data_loader is defined
        # Real data label: 1 (real)
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # Train Discriminator
        d_optimizer.zero_grad()
        real_data = real_data.to(device)
        d_real = discriminator(real_data)
        d_real_loss = criterion(d_real, real_labels)

        z = torch.randn(batch_size, 100).to(device)
        fake_data = generator(z)
        d_fake = discriminator(fake_data.detach())
        d_fake_loss = criterion(d_fake, fake_labels)

        d_loss = d_real_loss + d_fake_loss
        d_loss.backward()
        d_optimizer.step()

        # Train Generator
        g_optimizer.zero_grad()
        g_fake = discriminator(fake_data)
        g_loss = criterion(g_fake, real_labels)  # Wants the fake data to be labeled as real
        g_loss.backward()
        g_optimizer.step()

    print(f"Epoch [{epoch}/{epochs}], D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")


Epoch [1/10], D Loss: 0.6932, G Loss: 0.6931
Epoch [2/10], D Loss: 0.6905, G Loss: 0.6934
Epoch [3/10], D Loss: 0.6880, G Loss: 0.6938
Epoch [4/10], D Loss: 0.6851, G Loss: 0.6943
Epoch [5/10], D Loss: 0.6825, G Loss: 0.6947
Epoch [6/10], D Loss: 0.6799, G Loss: 0.6950
Epoch [7/10], D Loss: 0.6773, G Loss: 0.6953
Epoch [8/10], D Loss: 0.6746, G Loss: 0.6955
Epoch [9/10], D Loss: 0.6719, G Loss: 0.6957
Epoch [10/10], D Loss: 0.6692, G Loss: 0.6958



In [ ]:
# Save the trained generator model
torch.save(generator.state_dict(), '/3d_gan_generator.pth')
print("Model saved.")


Model saved.


In [ ]:
# Load the trained generator model
generator = Generator(z_dim=100).to(device)
generator.load_state_dict(torch.load('/3d_gan_generator.pth'))
generator.eval()


# Initialize the S3 client
s3_client = boto3.client('s3')

# Upload model to S3
s3_client.upload_file('/3d_gan_generator.pth', 'aws-model', 'models/3d_gan_generator.pth')
print("Model uploaded to S3.")


Model uploaded to S3.


In [ ]:
# Load your input obj file
input_obj = '/12221_Cat_v1_l3.obj'

# Convert to point cloud
input_pointcloud = load_obj_as_pointcloud(input_obj)

# Ensure the input is on the correct device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
input_pointcloud = input_pointcloud.to(device)

point_cloud = generated_data[0]  # Take the batch item
point_cloud = point_cloud.flatten()  # Flatten it for a point cloud format

# Create a point cloud and save
output = trimesh.PointCloud(point_cloud)
print(output)


# Aspose.3D Wavefront OBJ Exporter
# Copyright 2004-2024 Aspose Pty Ltd.
# File created: 11/09/2024 17:33:33

g Cat

#
# object Cat
#

v 0.6134 -21.9357 31.4441
v 0.6205 -21.8541 31.4935
v 0.5534 -21.8682 31.5159
v 0.5464 -21.9516 31.4643
v 0.6207 -21.761 31.5371
v 0.5544 -21.7724 31.5613
v 0.4816 -21.8847 31.531
v 0.4743 -21.9703 31.4781
v 0.4838 -21.7858 31.5776
v 0.6156 -21.6618 31.5749
v 0.607 -21.5619 31.6064
v 0.5439 -21.5675 31.633
v 0.5507 -21.6702 31.6005
v 0.4772 -21.5744 31.6514
v 0.4819 -21.6802 31.6178
v 0.332 -21.5913 31.6699
v 0.333 -21.7025 31.6343
v 0.4094 -21.6911 31.6286
v 0.4067 -21.5824 31.6631
v 0.3317 -21.8143 31.5926
v 0.4094 -21.8001 31.5875
v 0.3275 -21.9189 31.5447
v 0.3201 -22.0086 31.4907
v 0.3984 -21.9899 31.4865
v 0.4059 -21.9021 31.5402
v 0.329 -21.4886 31.6993
v 0.251 -21.4968 31.704
v 0.2529 -21.6008 31.6733
v 0.3267 -21.3918 31.7257
v 0.2497 -21.3991 31.7319
v 0.1694 -21.5046 31.7061
v 0.1705 -21.6094 31.6743
v 0.1687 -21.406 31.7352
v 0.3279 -21.2986

In [ ]:
import numpy as np
import trimesh
from scipy.spatial import cKDTree

# Function to load .obj file and sample point cloud from the surface
def load_obj_as_pointcloud(obj_file, num_points=10000):
    mesh = trimesh.load(obj_file)
    # Sample points from the surface of the mesh
    point_cloud, _ = trimesh.sample.sample_surface(mesh, num_points)
    return point_cloud

# Chamfer Distance calculation
def chamfer_distance_np(pc1, pc2):
    # Build KD trees for both point clouds
    kdtree_pc1 = cKDTree(pc1)
    kdtree_pc2 = cKDTree(pc2)

    # Find the nearest neighbors for each point in both point clouds
    dist_pc1_to_pc2, _ = kdtree_pc1.query(pc2)
    dist_pc2_to_pc1, _ = kdtree_pc2.query(pc1)

    # Chamfer distance is the sum of both directions
    chamfer_dist = np.mean(dist_pc1_to_pc2**2) + np.mean(dist_pc2_to_pc1**2)
    return chamfer_dist

# Load original and optimized obj files
original_obj = '/12221_Cat_v1_l3.obj'
optimized_obj = '/gan_12221_Cat_v1_l3.obj'

# Convert to point clouds
original_pointcloud = load_obj_as_pointcloud(original_obj)
optimized_pointcloud = load_obj_as_pointcloud(optimized_obj)

# Calculate Chamfer Distance between original and optimized point clouds
chamfer_dist = chamfer_distance_np(original_pointcloud, optimized_pointcloud)

# Print the Chamfer Distance
print(f"Chamfer Distance between original and optimized objects: {chamfer_dist}")


Chamfer Distance between original and optimized objects: 0.4646959352753117


### Evaluation


The Chamfer Distance measures how closely two point clouds match. It is typically used to compare the geometric similarity between 3D shapes, with lower values indicating better similarity between the two point clouds. The value of Chamfer Distance depends on factors like:

Interpreting Chamfer Distance:
Good Match:

Lower Chamfer Distance: The two 3D objects are geometrically close, with a value approaching 0 indicating near-perfect similarity. Typically, values under 1 or even lower (e.g., 0.1 to 0.5) indicate very good alignment, depending on the dataset and the number of points sampled.
Poor Match:

Higher Chamfer Distance: This indicates that the two objects differ more in shape. A large Chamfer distance could be around 1 or higher, depending on how large the differences are.
Ideal Values:

< 0.5: A good Chamfer Distance for most practical applications when comparing 3D meshes.
> 1.0: This could be a sign of noticeable differences in shape between the two meshes, but this depends on the resolution of the point cloud and the complexity of the shape.

In [ ]:
import numpy as np
import trimesh
from scipy.spatial import cKDTree

# Function to load .obj file and sample point cloud from the surface
def load_obj_as_pointcloud(obj_file, num_points=10000):
    mesh = trimesh.load(obj_file)
    # Sample points from the surface of the mesh
    point_cloud, _ = trimesh.sample.sample_surface(mesh, num_points)
    return point_cloud

# Chamfer Distance calculation
def chamfer_distance_np(pc1, pc2):
    # Build KD trees for both point clouds
    kdtree_pc1 = cKDTree(pc1)
    kdtree_pc2 = cKDTree(pc2)

    # Find the nearest neighbors for each point in both point clouds
    dist_pc1_to_pc2, _ = kdtree_pc1.query(pc2)
    dist_pc2_to_pc1, _ = kdtree_pc2.query(pc1)

    # Chamfer distance is the sum of both directions
    chamfer_dist = np.mean(dist_pc1_to_pc2**2) + np.mean(dist_pc2_to_pc1**2)
    return chamfer_dist

# Load original obj file
original_obj = '12221_Cat_v1_l3.obj'
original_pointcloud = load_obj_as_pointcloud(original_obj)

# Function to load model-generated output (for CNN, LSTM, GAN) as point cloud
def load_model_output(model_output_file, num_points=10000):
    return load_obj_as_pointcloud(model_output_file, num_points)

# Assuming the model outputs are saved as .obj files
# CNN output
cnn_output_obj = '/3d_cnn_12221_Cat_v1_l3.obj'
cnn_pointcloud = load_model_output(cnn_output_obj)

# LSTM output
lstm_output_obj = '/lstm_12221_Cat_v1_l3.obj'
lstm_pointcloud = load_model_output(lstm_output_obj)

# GAN output
gan_output_obj = '/gan_12221_Cat_v1_l3.obj'
gan_pointcloud = load_model_output(gan_output_obj)

# Calculate Chamfer Distance between original and model-generated point clouds
chamfer_dist_cnn = chamfer_distance_np(original_pointcloud, cnn_pointcloud)
chamfer_dist_lstm = chamfer_distance_np(original_pointcloud, lstm_pointcloud)
chamfer_dist_gan = chamfer_distance_np(original_pointcloud, gan_pointcloud)

# Print the Chamfer Distances
print(f"Chamfer Distance between original and CNN output: {chamfer_dist_cnn}")
print(f"Chamfer Distance between original and LSTM output: {chamfer_dist_lstm}")
print(f"Chamfer Distance between original and GAN output: {chamfer_dist_gan}")


Chamfer Distance between original and CNN output: 0.3682436665101978
Chamfer Distance between original and LSTM output: 0.1691736198922522
Chamfer Distance between original and GAN output: 0.4646959352753117
